In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
df = pd.read_csv('ufc_fights.csv')
# print(df.columns)
print(', '.join(df.columns))
# print(df.head())
# print(df.info())
# print(df.describe())

RedFighter, BlueFighter, RedOdds, BlueOdds, RedExpectedValue, BlueExpectedValue, Date, Location, Country, Winner, TitleBout, WeightClass, Gender, NumberOfRounds, BlueCurrentLoseStreak, BlueCurrentWinStreak, BlueDraws, BlueAvgSigStrLanded, BlueAvgSigStrPct, BlueAvgSubAtt, BlueAvgTDLanded, BlueAvgTDPct, BlueLongestWinStreak, BlueLosses, BlueTotalRoundsFought, BlueTotalTitleBouts, BlueWinsByDecisionMajority, BlueWinsByDecisionSplit, BlueWinsByDecisionUnanimous, BlueWinsByKO, BlueWinsBySubmission, BlueWinsByTKODoctorStoppage, BlueWins, BlueStance, BlueHeightCms, BlueReachCms, BlueWeightLbs, RedCurrentLoseStreak, RedCurrentWinStreak, RedDraws, RedAvgSigStrLanded, RedAvgSigStrPct, RedAvgSubAtt, RedAvgTDLanded, RedAvgTDPct, RedLongestWinStreak, RedLosses, RedTotalRoundsFought, RedTotalTitleBouts, RedWinsByDecisionMajority, RedWinsByDecisionSplit, RedWinsByDecisionUnanimous, RedWinsByKO, RedWinsBySubmission, RedWinsByTKODoctorStoppage, RedWins, RedStance, RedHeightCms, RedReachCms, RedWeightLb

In [19]:
#analyze and handle missing data

def missing_value_analysis(df):
    missing_values = df.isnull().sum()
    missing_values_percentage = (missing_values / len(df)) * 100
    missing_values_table = pd.concat([missing_values, missing_values_percentage], axis=1)
    missing_values_table = missing_values_table.rename(columns={0: 'Missing Values', 1: 'Percentage'})

    return missing_values_table[missing_values_table['Missing Values'] > 0].sort_values(by='Percentage', ascending=False)


missing_data = missing_value_analysis(df)
print(missing_data)

                       Missing Values  Percentage
BWFeatherweightRank              6527   99.984681
RWFeatherweightRank              6519   99.862132
BPFPRank                         6461   98.973652
BWFlyweightRank                  6455   98.881740
RWFlyweightRank                  6432   98.529412
BWStrawweightRank                6428   98.468137
BWBantamweightRank               6421   98.360907
BWelterweightRank                6409   98.177083
BBantamweightRank                6409   98.177083
BLightHeavyweightRank            6408   98.161765
BLightweightRank                 6408   98.161765
BFeatherweightRank               6404   98.100490
BFlyweightRank                   6398   98.008578
BMiddleweightRank                6391   97.901348
RWStrawweightRank                6382   97.763480
BHeavyweightRank                 6380   97.732843
RWBantamweightRank               6374   97.640931
RFeatherweightRank               6351   97.288603
RBantamweightRank                6347   97.227328


In [21]:
def create_rank_categories(df, rank_column):
    if rank_column not in df.columns or not pd.api.types.is_numeric_dtype(df[rank_column]):
        return None
    
    # Convert to float explicitly to handle NaN values correctly
    rank_values = df[rank_column].astype(float)
    
    # Create a mask for NaN values
    na_mask = rank_values.isna()
    
    # Initialize result series with proper size (using strings, not category yet)
    result = pd.Series(index=rank_values.index, dtype='object')
    
    # Assign categories manually using conditions instead of pd.cut to avoid date conversion
    result[(rank_values >= 0) & (rank_values <= 10)] = 'Top 10'
    result[(rank_values > 10) & (rank_values <= 25)] = 'Rank 11-25'  # Changed label to avoid date conversion
    result[(rank_values > 25) & (rank_values <= 50)] = 'Rank 26-50'  # Changed label format
    result[(rank_values > 50) & (rank_values <= 100)] = 'Rank 51-100'  # Changed label format
    result[(rank_values > 100) & (~na_mask)] = 'Rank 100+'  # Changed label format
    
    # Assign 'Unranked' only to actual NaN positions
    result[na_mask] = 'Unranked'
    
    # Create final categorical with all possible values
    all_categories = ['Top 10', 'Rank 11-25', 'Rank 26-50', 'Rank 51-100', 'Rank 100+', 'Unranked']
    return pd.Categorical(result, categories=all_categories)

In [22]:
# Handle missing values
def handle_missing_values(df):
   
    df_processed = df.copy()
    
    # 1.Drop EmptyArena column as it's not needed for meaningful analysis
    print(f"Dropping EmptyArena column as it's not needed for analysis")
    if 'EmptyArena' in df_processed.columns:
        df_processed = df_processed.drop('EmptyArena', axis=1)
    
    # 2.Handle FinishDetails (55.70% missing)
    print("Creating indicator for FinishDetails column")
    df_processed['FinishDetails_present'] = df_processed['FinishDetails'].notnull().astype(int)
    df_processed['FinishDetails_clean'] = df_processed['FinishDetails'].fillna("Not Recorded")
    
    # 3.Handle Match Weight Class Rank columns
    print("Creating indicators and categories for BMatchWCRank and RMatchWCRank")
    # For Blue corner
    df_processed['BMatchWCRank_present'] = df_processed['BMatchWCRank'].notnull().astype(int)
    # For Red corner
    df_processed['RMatchWCRank_present'] = df_processed['RMatchWCRank'].notnull().astype(int)
    
    # 4.Create ordinal categories for ranks
     # Create correct ordinal categories for ranks using the improved function
    df_processed['BMatchWCRank_cat'] = create_rank_categories(df_processed, 'BMatchWCRank')
    df_processed['RMatchWCRank_cat'] = create_rank_categories(df_processed, 'RMatchWCRank')
    
    # 5.Handle extreme missing values (>95%) - mostly weight class rankings
    extreme_missing_cols = missing_data[missing_data['Percentage'] > 95].index.tolist()
    print(f"Creating binary indicators for {len(extreme_missing_cols)} columns with >95% missing values")
    
    for col in extreme_missing_cols:
        df_processed[f"{col}_present"] = df_processed[col].notnull().astype(int)
    
    # 6.Create aggregated indicators for weight class rankings
    ranking_cols = [col for col in df_processed.columns if 'Rank' in col and col in extreme_missing_cols]
    
    # Aggregate by corner (blue/red)
    if ranking_cols:
        blue_rank_cols = [col for col in ranking_cols if col.startswith('B')]
        red_rank_cols = [col for col in ranking_cols if col.startswith('R')]
        
        if blue_rank_cols:
            df_processed['blue_has_any_rank'] = df_processed[blue_rank_cols].notnull().any(axis=1).astype(int)
        if red_rank_cols:
            df_processed['red_has_any_rank'] = df_processed[red_rank_cols].notnull().any(axis=1).astype(int)


     # 7. Handle columns with low data values missing (<21%)
    low_missing_cols = missing_data[missing_data['Percentage'] < 21].index.tolist()
    
    print(f"Standard imputation for {len(low_missing_cols)} columns with <23% missing values")
    
    for col in low_missing_cols:
        if col in df_processed.columns:
            # For numeric columns
            if pd.api.types.is_numeric_dtype(df_processed[col]):
                # Impute with median
                median_val = df_processed[col].median()
                df_processed[col] = df_processed[col].fillna(median_val)
                print(f"  - Imputed {col} with median: {median_val}")
            # For categorical/object columns
            else:
                # Impute with mode
                mode_val = df_processed[col].mode()[0] if not df_processed[col].mode().empty else "Unknown"
                df_processed[col] = df_processed[col].fillna(mode_val)
                print(f"  - Imputed {col} with mode: {mode_val}")       
    
    return df_processed

df_processed = handle_missing_values(df)
          

Dropping EmptyArena column as it's not needed for analysis
Creating indicator for FinishDetails column
Creating indicators and categories for BMatchWCRank and RMatchWCRank
Creating binary indicators for 26 columns with >95% missing values
Standard imputation for 25 columns with <23% missing values
  - Imputed BKOOdds with median: 525.0
  - Imputed BSubOdds with median: 1000.0
  - Imputed RSubOdds with median: 750.0
  - Imputed RKOOdds with median: 420.0
  - Imputed BlueDecOdds with median: 350.0
  - Imputed RedDecOdds with median: 250.0
  - Imputed BlueAvgSigStrLanded with median: 9.28
  - Imputed BlueAvgTDPct with median: 0.33
  - Imputed BlueAvgTDLanded with median: 1.0
  - Imputed BlueAvgSubAtt with median: 0.3
  - Imputed BlueAvgSigStrPct with median: 0.46
  - Imputed FinishRoundTime with mode: 5:00
  - Imputed FinishRound with median: 3.0
  - Imputed TotalFightTimeSecs with median: 900.0
  - Imputed RedAvgSigStrLanded with median: 15.6667
  - Imputed RedAvgTDPct with median: 0.338

In [23]:
# Verify results
print("\nVerifying results after processing")
missing_after = missing_value_analysis(df_processed.drop([col for col in df_processed.columns 
                                       if col.endswith('_present') or col.endswith('_cat') or col.endswith('_clean')], axis=1))
print("Original columns with missing values after processing:")
if len(missing_after) > 0:
    display(missing_after)
else:
    print("No missing values remain in processed original columns!")

if 'BMatchWCRank_cat' in df_processed.columns:
        print("\nBMatchWCRank_cat distribution:")
        print(df_processed['BMatchWCRank_cat'].value_counts(dropna=False))
    
if 'RMatchWCRank_cat' in df_processed.columns:
        print("\nRMatchWCRank_cat distribution:")
        print(df_processed['RMatchWCRank_cat'].value_counts(dropna=False))

# Save processed dataset
df_processed.to_csv('processed_fighting_data.csv', index=False)
print("\nProcessed dataset saved!")



Verifying results after processing
Original columns with missing values after processing:


,Missing Values,Percentage
BWFeatherweightRank,6527,99.984681
RWFeatherweightRank,6519,99.862132
BPFPRank,6461,98.973652
BWFlyweightRank,6455,98.881740
RWFlyweightRank,6432,98.529412
BWStrawweightRank,6428,98.468137
BWBantamweightRank,6421,98.360907
BWelterweightRank,6409,98.177083
BBantamweightRank,6409,98.177083
BLightHeavyweightRank,6408,98.161765



BMatchWCRank_cat distribution:
BMatchWCRank_cat
Unranked       5328
Top 10          784
Rank 11-25      416
Rank 26-50        0
Rank 51-100       0
Rank 100+         0
Name: count, dtype: int64

RMatchWCRank_cat distribution:
RMatchWCRank_cat
Unranked       4749
Top 10         1300
Rank 11-25      479
Rank 26-50        0
Rank 51-100       0
Rank 100+         0
Name: count, dtype: int64

Processed dataset saved!
